In [1]:
#to open cred files
import os
import yaml
#cleaning and operations
import pandas as pd
import numpy as np

In [2]:
#bigquery
import pandas_gbq
from google.cloud import bigquery

In [3]:
## got to service accounts in google cloud in project OAN-Nikarit and download json file
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='/Users/daniel/OAN/credentials/bigquery_oan_nikarit.json'

In [4]:
#FIREBASE
import firebase_admin
#Firbease (pip install firebase_admin)
from firebase_admin import credentials
from firebase_admin import firestore

In [10]:
filename='/Users/daniel/OAN/credentials/firebase_oan-nikarit.json'

In [11]:
#OPEN CONECTION TO FIREBASE
cred = credentials.Certificate(filename)
firebase_admin.initialize_app(cred)

db = firestore.client()

## Accounting table

### GET DATA

In [12]:
#get all the collection
col_query = db.collection('Accounting').stream()

In [13]:
#save it in a list of dictionaries
accounts = []
for acc in col_query:
    accounts.append(acc.to_dict())

In [14]:
#convert to dataframe
accounting = pd.DataFrame(accounts)

In [16]:
#nulls. size and datatype
accounting.info()

In [ ]:
#nulls heatmap - import seaborn as sns
#sns.heatmap(accounting.isna())

### CLEAN DATA

In [23]:
#lets see how it shows
#accounting.head()

#### convert execution to isostring

In [17]:
def clean_execution(x):
    if(len(x['execution_date'])<15):
        doc_id = x['id']
        exec_date=x['execution_date']+'T14:14:58.593Z'
        print(exec_date)
        print(len(x['execution_date']))
        #upload to firebase if whished
        #a=db.collection(u'Accounting').document(doc_id).update({'execution_date':exec_date})
        
        return exec_date
    else:
        return x['execution_date']
        

In [18]:
accounting['execution_date']=accounting.apply(lambda x: clean_execution(x) ,axis=1)

##### clean VAT and replace nan with 0

In [19]:
def clean_vat_amount(x):
    doc_id = x['id']
    amount=x['amount']
    origin = x['origin']
    if origin =='script_dani':
        if x['vat']>0:
            vat=21
        else:
            vat=0
        #a=db.collection(u'Accounting').document(doc_id).update({'vat':vat})
    else:
        vat=x['vat']
        if not(vat >=0):
            vat=0
    vat_amount=amount*(1-1/(1+vat/100))

    #a=db.collection(u'Accounting').document(doc_id).update({'vat_amount':vat_amount})

    return pd.Series({'vat':vat,'vat_amount':vat_amount})

In [20]:
accounting[['vat','vat_amount']] = accounting.apply(lambda x: clean_vat_amount(x),axis=1)

###### convert dates to datetime

In [21]:
#utc=True important if dates are in diferent formats
accounting['creation_date'] = pd.to_datetime(accounting['creation_date'],utc=True)
accounting['execution_date'] = pd.to_datetime(accounting['execution_date'],utc=True)

In [58]:
def getimages(x):
    if(len(x)==0):
        return ''
    elif('download_url' in x[0].keys()):
        download_url = x[0]['download_url']
        if(download_url == None):
            return ''
        else:
            return download_url
    else:
        return ''

In [59]:
accounting['receipt']=accounting.images.apply(lambda x: getimages(x))

##### double check

In [61]:
accounting.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1438 entries, 0 to 1437
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   creation_date   1438 non-null   datetime64[ns, UTC]
 1   concept         1438 non-null   object             
 2   description     1422 non-null   object             
 3   user_in_charge  1438 non-null   object             
 4   intervention    1438 non-null   object             
 5   amount          1438 non-null   float64            
 6   account_id      1438 non-null   object             
 7   phase           409 non-null    object             
 8   project         1438 non-null   object             
 9   id              1438 non-null   object             
 10  creator_user    1438 non-null   object             
 11  place           1438 non-null   object             
 12  images          1438 non-null   object             
 13  target_id       1438 non-null   o

### select columns

In [62]:
#and the order whe want to upload to bigquery
df = accounting[['concept', 'place', 'creation_date', 'vat', 'description', 'amount',
       'id', 'phase', 'creator_user', 'execution_date',
       'user_in_charge', 'origin', 'account_id', 'project', 'code', 'type',
       'intervention', 'target_id', 'vat_amount', 'receipt']]

## upload to BIGQUERY

In [63]:
client = bigquery.Client()

In [64]:
dataset_id = "{}.contoan".format(client.project)

In [65]:
dataset = bigquery.Dataset(dataset_id)

In [66]:
table_id = "{}.accounting".format(dataset.dataset_id)

pandas_gbq.to_gbq(df, table_id, project_id=client.project, if_exists='replace')

1it [00:04,  4.31s/it]


##### september 2020 backup

In [67]:
table_id = "{}.accounting_202009".format(dataset.dataset_id)

pandas_gbq.to_gbq(df, table_id, project_id=client.project, if_exists='replace')

1it [00:04,  4.48s/it]


## get data

In [ ]:
sql = """
    SELECT *
    FROM `oan-nikarit.contoan.accounting`
    LIMIT 100
"""

# Run a Standard SQL query using the environment's default project
df = pd.read_gbq(sql, dialect='standard')

### SUBIR VENTAS

#### get data

In [ ]:
col_query = db.collection('Nikarit_Sales').stream()

In [ ]:
ventas = []
for acc in col_query:
    ventas.append(acc.to_dict())

In [ ]:
df_ventas = pd.DataFrame(ventas)

### view

In [ ]:
df_ventas.info()

#### transform data

In [ ]:
cambios = ['subtotal_price','balsamo_unit_price','packContigo_unit_price','crema_unit_price','packHidra_unit_price',
          'manteca_unit_price','buyer_accepts_marketing']

for cambio in cambios:
    df_ventas[cambio]=pd.to_numeric(df_ventas[cambio])

In [ ]:
df_ventas['creation_date']=pd.to_datetime(df_ventas['creation_date'],utc=True)
df_ventas['close_date']=pd.to_datetime(df_ventas['close_date'],utc=True)

In [69]:
#double check
df_ventas.info()

### METER MUJERES Y HOMBRES

In [ ]:
#client_gender

In [ ]:
#lista de mujeres
url = 'https://raw.githubusercontent.com/marcboquet/spanish-names/master/mujeres.csv'
df_mujeres = pd.read_csv(url, error_bad_lines=False)
l_mujeres = df_mujeres['nombre'].to_list()

In [ ]:
l_mujeres = [str(nom).lower() for nom in l_mujeres]

In [ ]:
#lista de hombres
url = 'https://raw.githubusercontent.com/marcboquet/spanish-names/master/hombres.csv'
df_hombres = pd.read_csv(url, error_bad_lines=False)
l_hombres = df_hombres['nombre'].to_list()

In [ ]:
l_hombres = [nom.lower() for nom in l_hombres]

In [ ]:
df_ventas['client_name'].value_counts()

In [ ]:
#aplicar sexo
df_ventas['client_gender'] = df_ventas.apply(lambda x: 
                            'F' if unidecode.unidecode(
                                    str(x['client_name']).replace(" ", "").lower()) in l_mujeres
                            else 'M' if unidecode.unidecode(
                                str(x['client_name']).replace(" ", "").lower())  in l_hombres
                            else 'F' if unidecode.unidecode(
                                str(x['client_name']).split(" ")[0].lower())  in l_mujeres
                            else 'M' if unidecode.unidecode(
                                str(x['client_name']).split(" ")[0].lower())  in l_hombres

                            else 'N/A' , axis =1)

In [ ]:
#ver que sale
df_ventas['client_gender'].value_counts() # A MEJORRAR

In [ ]:
#subir a bigquery
table_id = "contoan.sales"

pandas_gbq.to_gbq(df_ventas, table_id, project_id=client.project, if_exists='replace')